In [1]:
import mysql.connector
import pandas as pd
from insert_data import get_subtask_to_index_dict
from insert_data import create_injection_df, check_to_insert_or_upate
from huggingface_hub import HfApi, list_models
import pandas as pd
import json
import numpy as np
from tqdm import tqdm
#Create the connection object   
myconn = mysql.connector.connect(host = "172.17.101.178", 
                                 user = "root",
                                 passwd = "2CWTL9jREpp%WbqS", 
                                 port = 3326,
                                 database = "hf_model2")  
    
cursor = myconn.cursor(buffered=True)

/home/poon/virtual_env/first_project/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [41]:
cursor.reset()
myconn.close()

In [2]:
subtask_to_code_dict = get_subtask_to_index_dict(cursor=cursor)

Getting subtask dictionary to index
Successfully get dictionary to index
{'text-to-speech': 1, 'audio-to-audio': 2, 'voice-activity-detection': 3, 'image-classification': 4, 'image-segmentation': 5, 'object-detection': 6, 'video-classification': 7, 'zero-shot-image-classification': 8, 'image-to-image': 9, 'unconditional-image-generation': 10, 'feature-extraction': 11, 'text-to-image': 12, 'visual-question-answering': 13, 'document-question-answering': 14, 'image-to-text': 15, 'graph-ml': 16, 'text-to-video': 17, 'fill-mask': 18, 'question-answering': 19, 'text-generation': 20, 'text-classification': 21, 'translation': 22, 'token-classification': 23, 'text2text-generation': 24, 'conversational': 25, 'sentence-similarity': 26, 'summarization': 27, 'zero-shot-classification': 28, 'table-question-answering': 29, 'time-series-forecasting': 30, 'other': 31, 'reinforcement-learning': 32, 'robotics': 33, 'tabular-classification': 34, 'tabular-regression': 35, 'None': 36}


In [3]:
models = list_models(full= True,
                     cardData = True)

In [4]:
metadata_cols = ['_id', 'tags', 'cardData', 'private' ]

In [3]:
original_df = pd.read_csv('/home/poon/work/20230906_original_df.csv', index_col=0)


In [5]:
all_insert_query = """
    insert into modelheader (modelname, modelsourcetypecode, modelsubtasktypecode,
       modelmodifieddate, author, library_name, numberlikes,
       numberdownloads, metadata, createddate, createdby)


    value (%s, %s, %s, %s, %s, %s, %s, %s, %s, current_timestamp(), 'Poon')
"""


all_update_query = """
    update modelheader
    set  numberlikes =  %s,  numberdownloads =  %s,  updateddate = current_timestamp(), updatedby = %s
    where modelname = %s
    """

In [12]:
chunk_step = 1000
start_idx = 0
stop_idx = 10000

for i in tqdm(np.arange(start=start_idx, stop=stop_idx, step = chunk_step)):
    # print(i, i + chunk_step)
    convert_df = create_injection_df(model_df= original_df.iloc[i : i +chunk_step ],
                                     subtask_to_code_dict=subtask_to_code_dict)
    for data in [tuple(x) for x in convert_df.to_records(index=False)]:
    # print(data[0], int(data[1]), int(data[2]))

        cursor.execute("SELECT * FROM modelheader WHERE modelname = %s", (data[0],))
        existing_row = cursor.fetchone()
        # print(cursor.fetchone())
        if existing_row:
            # Row exists, update it3
            # print('alreadu have')
            cursor.execute(all_update_query, (int(data[6]), int(data[7]), 'Poon', data[0]))
        else:
            # Row does not exist, insert it
            # print("not having")
            # print(data)
            cursor.execute(all_insert_query, (data[0], int(data[1]), int(data[2]), data[3], data[4], data[5], int(data[6]), int(data[7]), json.dumps(data[8])))
            
    myconn.commit()
        


  0%|          | 0/10 [00:00<?, ?it/s]

 20%|██        | 2/10 [04:16<17:06, 128.27s/it]


KeyboardInterrupt: 

In [8]:
convert_df

,modelname,modelsourcetypecode,modelsubtasktypecode,modelmodifieddate,author,library_name,numberlikes,numberdownloads,metadata
59000,yanaiela/roberta-base-epoch_18,1,18,2022-07-29T22:46:26.000Z,yanaiela,transformers,0,15,"{""_id"": ""62e2c161555a866437a920fe"", ""tags"": ""[..."
59001,Vlasta/DNADebertaSentencepiece30k,1,18,2022-07-31T12:30:51.000Z,Vlasta,transformers,0,0,"{""_id"": ""62e2c17e3a616d16e2f7944f"", ""tags"": ""[..."
59002,yanaiela/roberta-base-epoch_19,1,18,2022-07-29T22:46:46.000Z,yanaiela,transformers,0,15,"{""_id"": ""62e2c197ebcbb4b2b5d0af1a"", ""tags"": ""[..."
59003,yanaiela/roberta-base-epoch_20,1,18,2022-07-29T22:47:06.000Z,yanaiela,transformers,0,15,"{""_id"": ""62e2c1c7a133a252b5d0fe42"", ""tags"": ""[..."
59004,yanaiela/roberta-base-epoch_21,1,18,2022-07-29T22:47:23.000Z,yanaiela,transformers,0,15,"{""_id"": ""62e2c1f92be89f0bf5d69b29"", ""tags"": ""[..."
...,...,...,...,...,...,...,...,...,...
59095,AbidHasan95/movieHunt3-ner,1,23,2022-07-29T08:36:50.000Z,AbidHasan95,transformers,0,0,"{""_id"": ""62e30011c4a0c0b9a0cc0c02"", ""tags"": ""[..."
59096,Evelyn18/roberta-base-spanish-squades-becasInc...,1,99,2022-07-28T21:45:14.000Z,Evelyn18,NaN,0,0,"{""_id"": ""62e3036ab3bfe4068810120d"", ""tags"": ""[..."
59097,ICML2022/Tranception,1,18,2022-07-28T23:28:37.000Z,ICML2022,transformers,5,2,"{""_id"": ""62e30d6b3a616d16e2fa29bc"", ""tags"": ""[..."
59098,domenicrosati/deberta-v3-large-finetuned-DAGPa...,1,21,2022-07-30T09:31:40.000Z,domenicrosati,transformers,0,0,"{""_id"": ""62e3127eb3bfe406881092d8"", ""tags"": ""[..."


In [28]:
all_insert_query = """
    insert into modelheader (modelname, modelsourcetypecode, modelsubtasktypecode,
       modelmodifieddate, author, library_name, numberlikes,
       numberdownloads, metadata, createddate, createdby)


    value (%s, %s, %s, %s, %s, %s, %s, %s, %s, current_timestamp(), 'Poon')
"""


all_update_query = """
    update modelheader
    set  numberlikes =  %s,  numberdownloads =  %s,  updateddate = current_timestamp(), updatedby = %s
    where modelname = %s
    """

In [16]:
convert_df.head()

,modelname,modelsourcetypecode,modelsubtasktypecode,modelmodifieddate,author,library_name,numberlikes,numberdownloads,metadata
0,albert-base-v1,1,18,2023-04-06T13:42:57.000Z,NaN,transformers,3,52846,"{""_id"": ""621ffdc036468d709f174328"", ""tags"": ""[..."
1,albert-base-v2,1,18,2023-05-30T07:52:10.000Z,NaN,transformers,60,7162784,"{""_id"": ""621ffdc036468d709f174329"", ""tags"": ""[..."
2,albert-large-v1,1,18,2021-01-13T15:29:06.000Z,NaN,transformers,0,509,"{""_id"": ""621ffdc036468d709f17432a"", ""tags"": ""[..."
3,albert-large-v2,1,18,2023-04-06T13:41:50.000Z,NaN,transformers,11,13050,"{""_id"": ""621ffdc036468d709f17432b"", ""tags"": ""[..."
4,albert-xlarge-v1,1,18,2023-08-11T07:30:41.000Z,NaN,transformers,0,456,"{""_id"": ""621ffdc036468d709f17432c"", ""tags"": ""[..."


In [19]:
for data in [tuple(x) for x in convert_df.to_records(index=False)]:
    # print(data[0], int(data[1]), int(data[2]))

    cursor.execute("SELECT * FROM modelheader WHERE modelname = %s", (data[0],))
    existing_row = cursor.fetchone()
    # print(cursor.fetchone())
    if existing_row:
        # Row exists, update it3
        print('alreadu have')
        # cursor.execute(all_update_query, ('Poon', data[0]))
    else:
        # Row does not exist, insert it
        print("not having")
        # print(data)
        cursor.execute(all_insert_query, (data[0], int(data[1]), int(data[2]), data[3], data[4], data[5], int(data[6]), int(data[7]), json.dumps(data[8])))
        
myconn.commit()

alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
alreadu have
not having
not having
not having
not having
not having
not having
not having
not having
not having
not having
not having
not having
not having
not having
not having
not having
not having
not having
not having
not having
not having
not having
not having
not having
not having
not having
not having
not having
not having
not having
not having
not havin

In [18]:
convert_df

,modelname,modelsourcetypecode,modelsubtasktypecode,modelmodifieddate,author,library_name,numberlikes,numberdownloads,metadata
0,albert-base-v1,1,18,2023-04-06T13:42:57.000Z,NaN,transformers,3,52846,"{""_id"": ""621ffdc036468d709f174328"", ""tags"": ""[..."
1,albert-base-v2,1,18,2023-05-30T07:52:10.000Z,NaN,transformers,60,7162784,"{""_id"": ""621ffdc036468d709f174329"", ""tags"": ""[..."
2,albert-large-v1,1,18,2021-01-13T15:29:06.000Z,NaN,transformers,0,509,"{""_id"": ""621ffdc036468d709f17432a"", ""tags"": ""[..."
3,albert-large-v2,1,18,2023-04-06T13:41:50.000Z,NaN,transformers,11,13050,"{""_id"": ""621ffdc036468d709f17432b"", ""tags"": ""[..."
4,albert-xlarge-v1,1,18,2023-08-11T07:30:41.000Z,NaN,transformers,0,456,"{""_id"": ""621ffdc036468d709f17432c"", ""tags"": ""[..."
...,...,...,...,...,...,...,...,...,...
95,17luke/wav2vec2-large-xls-r-300m-icelandic-sam...,1,99,2022-02-16T20:09:30.000Z,17luke,NaN,0,0,"{""_id"": ""621ffdc036468d709f174397"", ""tags"": ""[..."
96,18811449050/bert_cn_finetuning,1,21,2021-05-18T17:03:47.000Z,18811449050,transformers,0,13,"{""_id"": ""621ffdc036468d709f174399"", ""tags"": ""[..."
97,18811449050/bert_finetuning_test,1,21,2021-05-18T17:05:20.000Z,18811449050,transformers,0,25,"{""_id"": ""621ffdc036468d709f17439a"", ""tags"": ""[..."
98,1Basco/DialoGPT-small-jake,1,25,2021-09-22T03:32:39.000Z,1Basco,transformers,0,21,"{""_id"": ""621ffdc036468d709f17439c"", ""tags"": ""[..."
